In [45]:
import pandas as pd
import numpy as np
import pickle
import math
from math import cos, sin, pi
import matplotlib.pyplot as plt

In [46]:
def dist_(x1,y1,x2,y2):
  try:
    x = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
  except:
    x = 0
  return x

### Exploring IBTrACS data - what cyclones have radii


In [47]:
df2 = pd.read_csv('IBTrACS_original.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,21,22,61,66,128,143,148) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
df2.head()

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,REU_GUSTP,USA_SEAHGT,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min
0,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 03:00:00,NR,10.90,80.30,...,NaN,NaN,NaN,9,266,1842,10,25,3,0
1,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 06:00:00,NR,10.87,79.83,...,NaN,NaN,NaN,9,267,1842,10,25,6,0
2,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 09:00:00,NR,10.84,79.35,...,NaN,NaN,NaN,9,267,1842,10,25,9,0
3,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 12:00:00,NR,10.82,78.88,...,NaN,NaN,NaN,9,267,1842,10,25,12,0
4,1842298N11080,1842,1,NI,BB,NOT_NAMED,1842-10-25 15:00:00,NR,10.80,78.40,...,NaN,NaN,NaN,9,268,1842,10,25,15,0


In [49]:
df2['TOK_R50_L']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
669865   NaN
669866   NaN
669867   NaN
669868   NaN
669869   NaN
Name: TOK_R50_L, Length: 669870, dtype: float64

In [50]:
df2['LON_1'] = df2.groupby(['SID'])['LON'].shift(-1)
df2['LAT_1'] = df2.groupby(['SID'])['LAT'].shift(-1)

In [51]:
def big_wind(row):
  x1 = row['LAT']
  x2 = row['LAT_1']
  y1 = row['LON']
  y2 = row['LON_1']

  return dist_(x1,y1,x2,y2)
df2['dist'] = df2.apply(big_wind, axis=1)
df2['ISO_TIME'] = pd.to_datetime(df2['ISO_TIME'])
df2["rank"] = df2.groupby("SID")["ISO_TIME"].rank("dense", ascending=False)
df2["rank_1"] = df2.groupby("SID")["ISO_TIME"].rank("dense", ascending=True)

In [52]:
#df2['NEW_D_34'] = [df2['BOM_R34_NE']==df2['BOM_R34_NE']].NAME.unique()

In [53]:
for j in [34,50,64]:
  df2['max_'+str(j)] = df2[['USA_R'+str(j)+'_NE', 'USA_R'+str(j)+'_SE', 'USA_R'+str(j)+'_SW', 'USA_R'+str(j)+'_NW',
                            'REU_R'+str(j)+'_NE', 'REU_R'+str(j)+'_SE', 'REU_R'+str(j)+'_SW', 'REU_R'+str(j)+'_NW',
                            'BOM_R'+str(j)+'_NE', 'BOM_R'+str(j)+'_SE', 'BOM_R'+str(j)+'_SW', 'BOM_R'+str(j)+'_NW']].max(axis=1)  #USA_R34_NE

In [54]:
df2['max_50'] = df2[['max_50', 'TOK_R50_L']].max(axis=1)  #USA_R34_NE

In [55]:
df2['RMW'] = df2[['USA_RMW', 'REU_RMW', 'BOM_RMW']].max(axis=1)  #USA_R34_NE

In [56]:
df4 = df2[df2['max_34']==df2['max_34']][['NAME','SEASON']]

In [57]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57138 entries, 472432 to 669868
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NAME    57138 non-null  object
 1   SEASON  57138 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [58]:
df4 = df4.drop_duplicates()

In [59]:
len(df4)

1418

In [60]:
df4['in_tracks'] = True

### Estimating Radiouses of 34 50 60 kn winds 

In [61]:
df2['WMO_PRES_delta'] = df2.groupby(['SID'])['WMO_PRES'].diff(periods=-1)
df2.groupby('SID')['WMO_PRES_delta'].fillna(df2.median().iloc[0])

0         1969.0
1         1969.0
2         1969.0
3         1969.0
4         1969.0
           ...  
669865    1969.0
669866    1969.0
669867    1969.0
669868    1969.0
669869    1969.0
Name: WMO_PRES_delta, Length: 669870, dtype: float64

In [62]:
df2.groupby('SID')['dist'].fillna(df2.median().iloc[0])

0            0.470956
1            0.480937
2            0.470425
3            0.480416
4            0.480104
             ...     
669865       0.257099
669866       0.351710
669867       0.414849
669868       0.451221
669869    1969.000000
Name: dist, Length: 669870, dtype: float64

In [63]:
df2['dist'].fillna(df2.median().iloc[0])

0            0.470956
1            0.480937
2            0.470425
3            0.480416
4            0.480104
             ...     
669865       0.257099
669866       0.351710
669867       0.414849
669868       0.451221
669869    1969.000000
Name: dist, Length: 669870, dtype: float64

In [64]:
j#more 'BASIN','SUBBASIN' 'TRACK_TYPE' 'DIST2LAND','LANDFALL', 'STORM_DR'
#

64

In [65]:
df2['BASIN'] = df2['BASIN'].fillna('NAM')

In [66]:
ib = df2[df2['max_64']==df2['max_64']][['NAME','SEASON', 'SID', 'WMO_PRES_delta',
                                                                                                   'NUMBER',	'BASIN','SUBBASIN','NAME','ISO_TIME','NATURE','LAT','LON','WMO_WIND','WMO_PRES',
                                                                                                   	'TRACK_TYPE','DIST2LAND','LANDFALL',
                                                                                                   'STORM_SPD', 'dist',	'STORM_DR',	'year',	'month',	'day',	'hour', 'min', 'max_34', 'max_50', 'max_64', 'rank', 'rank_1'
                                                                                                   
                                                                                                   
                                                                                                   ]]

In [67]:
ib

,NAME,SEASON,SID,WMO_PRES_delta,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,...,year,month,day,hour,min,max_34,max_50,max_64,rank,rank_1
495848,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 12:00:00,TS,...,1992,3,13,12,0,NaN,NaN,70.0,60.0,77.0
495849,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 15:00:00,TS,...,1992,3,13,15,0,NaN,NaN,70.0,59.0,78.0
495850,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 18:00:00,TS,...,1992,3,13,18,0,NaN,NaN,70.0,58.0,79.0
495851,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 21:00:00,TS,...,1992,3,13,21,0,NaN,NaN,60.0,57.0,80.0
495852,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-14 00:00:00,TS,...,1992,3,14,0,0,NaN,NaN,50.0,56.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667951,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 03:00:00,TS,...,2018,10,31,3,0,150.0,69.0,29.0,5.0,34.0
667952,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 06:00:00,TS,...,2018,10,31,6,0,150.0,69.0,29.0,4.0,35.0
667953,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 09:00:00,TS,...,2018,10,31,9,0,202.0,75.0,32.0,3.0,36.0
667954,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 12:00:00,TS,...,2018,10,31,12,0,277.0,81.0,35.0,2.0,37.0


In [68]:
#ib = pd.get_dummies(ib, columns=['BASIN','SUBBASIN', 'WMO_AGENCY','TRACK_TYPE',  'NATURE'])

In [69]:
ib

,NAME,SEASON,SID,WMO_PRES_delta,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,...,year,month,day,hour,min,max_34,max_50,max_64,rank,rank_1
495848,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 12:00:00,TS,...,1992,3,13,12,0,NaN,NaN,70.0,60.0,77.0
495849,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 15:00:00,TS,...,1992,3,13,15,0,NaN,NaN,70.0,59.0,78.0
495850,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 18:00:00,TS,...,1992,3,13,18,0,NaN,NaN,70.0,58.0,79.0
495851,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 21:00:00,TS,...,1992,3,13,21,0,NaN,NaN,60.0,57.0,80.0
495852,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-14 00:00:00,TS,...,1992,3,14,0,0,NaN,NaN,50.0,56.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667951,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 03:00:00,TS,...,2018,10,31,3,0,150.0,69.0,29.0,5.0,34.0
667952,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 06:00:00,TS,...,2018,10,31,6,0,150.0,69.0,29.0,4.0,35.0
667953,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 09:00:00,TS,...,2018,10,31,9,0,202.0,75.0,32.0,3.0,36.0
667954,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 12:00:00,TS,...,2018,10,31,12,0,277.0,81.0,35.0,2.0,37.0


In [70]:
# Code modified by Raghuram on 6th April to fill the missing values with median
cols = ['WMO_WIND','WMO_PRES']
ib[cols] = ib.groupby('SID')[cols].ffill().fillna(ib[cols].median()).astype(int)

In [71]:
#ib['RMW_1'] = ib.groupby(['SID'])['RMW'].shift(-1)

In [72]:
# Code modified by Raghuram on 6th April to fill the missing values with median
cols_winds = ['max_34','max_50','max_64', 'dist']     
ib[cols] = ib.groupby('SID')[cols].ffill().fillna(ib[cols].median()).astype(int)

In [73]:
ib

,NAME,SEASON,SID,WMO_PRES_delta,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,...,year,month,day,hour,min,max_34,max_50,max_64,rank,rank_1
495848,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 12:00:00,TS,...,1992,3,13,12,0,NaN,NaN,70.0,60.0,77.0
495849,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 15:00:00,TS,...,1992,3,13,15,0,NaN,NaN,70.0,59.0,78.0
495850,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 18:00:00,TS,...,1992,3,13,18,0,NaN,NaN,70.0,58.0,79.0
495851,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-13 21:00:00,TS,...,1992,3,13,21,0,NaN,NaN,60.0,57.0,80.0
495852,FRAN,1992,1992064S10184,NaN,23,SP,EA,FRAN,1992-03-14 00:00:00,TS,...,1992,3,14,0,0,NaN,NaN,50.0,56.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667951,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 03:00:00,TS,...,2018,10,31,3,0,150.0,69.0,29.0,5.0,34.0
667952,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 06:00:00,TS,...,2018,10,31,6,0,150.0,69.0,29.0,4.0,35.0
667953,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 09:00:00,TS,...,2018,10,31,9,0,202.0,75.0,32.0,3.0,36.0
667954,OSCAR,2018,2018300N26315,NaN,97,NAM,NaN,OSCAR,2018-10-31 12:00:00,TS,...,2018,10,31,12,0,277.0,81.0,35.0,2.0,37.0


In [74]:
df2['BASIN'].unique()

array(['NI', 'SI', 'NAM', 'EP', 'WP', 'SP', 'SA'], dtype=object)

In [75]:
ib.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17093 entries, 495848 to 667955
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   NAME            17093 non-null  object        
 1   SEASON          17093 non-null  int64         
 2   SID             17093 non-null  object        
 3   WMO_PRES_delta  1975 non-null   float64       
 4   NUMBER          17093 non-null  int64         
 5   BASIN           17093 non-null  object        
 6   SUBBASIN        14043 non-null  object        
 7   NAME            17093 non-null  object        
 8   ISO_TIME        17093 non-null  datetime64[ns]
 9   NATURE          17093 non-null  object        
 10  LAT             17093 non-null  float64       
 11  LON             17093 non-null  float64       
 12  WMO_WIND        17093 non-null  int64         
 13  WMO_PRES        17093 non-null  int64         
 14  TRACK_TYPE      17093 non-null  object        
 

In [76]:
len(ib.dropna())

1921

In [77]:
ib = pd.get_dummies(ib, columns=['BASIN'])

In [78]:
ib = pd.get_dummies(ib, columns=['month']) 

In [79]:
ib['dist'] = ib['dist'].fillna(method = 'ffill')

In [80]:
import numpy as np
from sklearn.model_selection import train_test_split
column_names = ['LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND','LANDFALL','STORM_SPD',	
                'STORM_DR', 'dist',	
                'month_1',	'month_2',	'month_3',	'month_4',	'month_5',	'month_6',
                'month_7',	'month_8',	'month_9',	'month_10',	'month_11',	'month_12',
                'rank', 'rank_1',
                'BASIN_EP','BASIN_NI','BASIN_SI','BASIN_SP','BASIN_WP']
X = ib[column_names]
y = ib['max_64']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [81]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#model.fit(X, y)
# Code modified by Raghuram on 6th April to avoid data leakage
model.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [82]:
y_pred = model.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 11.31032827323686
Mean Squared Error: 280.5023115984718
Root Mean Squared Error: 16.74820323492857


In [84]:
df66 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df68 = df66.head(25)

In [85]:
df66

,Actual,Predicted
636527,30.0,30.987662
646984,30.0,26.701166
597792,25.0,27.102476
626752,25.0,25.846843
651230,40.0,42.198704
...,...,...
653398,30.0,28.822999
662619,95.0,47.674723
649762,40.0,31.375833
639083,35.0,31.565439


### Model - Random Forest Regressor

In [86]:
# Code for 'Model - Random Forest Regressor' added by Raghuram on 6th April 2020
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_estimators = 60, random_state = 0)
model2.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=60, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [87]:
y_pred = model2.predict(X_test)

In [88]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 4.594932931513325
Mean Squared Error: 57.20207818747661
Root Mean Squared Error: 7.5632055497306565


**Observation**: With this RandomForestRegressor model, MAE reduced from 51 to 20.9 and RMSE reduced from 215 to 104 when compared with Linear Regression model.

In [89]:
df_rf = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_rf.head(125)

,Actual,Predicted
636527,30.0,22.533333
646984,30.0,31.616667
597792,25.0,33.633333
626752,25.0,21.566667
651230,40.0,40.433333
...,...,...
643641,20.0,21.433333
634035,40.0,31.833333
612694,32.0,28.800000
606902,40.0,46.616667


In [90]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 300, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [50, 77, 105, 133, 161, 188, 216, 244, 272, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [91]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random_2 = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random_2.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [92]:
rf_random_2.best_params_

{'n_estimators': 133,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [93]:
y_pred = rf_random_2.predict(X_test)

In [94]:
y_pred_2 = rf_random_2.predict(X_test)

In [97]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_2))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_2))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_2)))

Mean Absolute Error: 3.897880305388623
Mean Squared Error: 40.71500735990161
Root Mean Squared Error: 6.380831243646991


AttributeError: 'LinearRegression' object has no attribute 'feature_importances_'

In [ ]:
from google.colab import files
files.download('rf_random_64.sav') 

In [ ]:
from google.colab import files
files.download('max_64.csv') 

##Populating with predictions 

In [98]:
filename = 'rf_random_64.sav'
pickle.dump(rf_random_2, open(filename, 'wb'))
 



In [ ]:
# load the model from disk
rf_random_64 = pickle.load(open('rf_random_64.sav', 'rb'))
result = rf_random_64.score(X_test, y_test)
print(result)

In [61]:
pip install rfpimp

Note: you may need to restart the kernel to use updated packages.


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   24.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   24.1s finished


In [101]:
perm_imp_rfpimp

,Importance
Feature,
LAT,0.337345
WMO_PRES,0.301884
rank_1,0.135092
LON,0.104093
WMO_WIND,0.100090
month_9,0.073521
STORM_DR,0.060482
dist,0.056849
rank,0.050699


In [ ]:
rf_random_64

In [99]:
pre_df  = pd.read_csv('OUTPUT_WBI_cyclones.csv', sep = ';', low_memory=False)

In [100]:
pre_df_unique  = pre_df[['NAME',	'Year']]

In [101]:
pre_df_unique = pre_df_unique.drop_duplicates()

In [102]:
pre_df['in_pre'] = True

In [103]:
df2['in_ibtrax'] = True

In [104]:
df = pd.merge(df2, pre_df, how='outer', left_on = ['NAME', 'SEASON'], right_on=['NAME','Year'], suffixes=('', '_pre'))

In [105]:
df = pd.get_dummies(df, columns=['BASIN','month'])

In [106]:
#y = df['max_50'][df['max_50']!=df['max_50']]

In [107]:
df['WMO_WIND'] =df.groupby('SID')['WMO_WIND'].fillna(df['WMO_WIND'].median())
df['WMO_WIND'] = df['WMO_WIND'].fillna(df['WMO_WIND'].median())
df['WMO_PRES']=df.groupby('SID')['WMO_PRES'].fillna(df['WMO_PRES'].median())
df['WMO_PRES'] = df['WMO_PRES'].fillna(df['WMO_PRES'].median())
df['STORM_SPD']=df.groupby('SID')['STORM_SPD'].fillna(df['STORM_SPD'].median())
df['STORM_SPD'] = df['STORM_SPD'].fillna(df['STORM_SPD'].median())
df['STORM_SPD'] = df['STORM_SPD'].fillna(df['STORM_SPD'].median())
df['STORM_DR']=df.groupby('SID')['STORM_DR'].fillna(df['STORM_DR'].median())
df['STORM_DR']=df.groupby('SID')['STORM_DR'].fillna(df['STORM_DR'].median())

In [108]:
df['dist'] = df['dist'].fillna(method='ffill')

##TESTING ACCURACY ON NEW DATA

In [135]:

import numpy as np
from sklearn.model_selection import train_test_split
column_names = ['LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND','LANDFALL','STORM_SPD',	
                'STORM_DR', 'dist',	
                'month_1.0',	'month_2.0',	'month_3.0',	'month_4.0',	'month_5.0',	'month_6.0',
                'month_7.0',	'month_8.0',	'month_9.0',	'month_10.0',	'month_11.0',	'month_12.0',
                'rank', 'rank_1',
                'BASIN_EP','BASIN_NI','BASIN_SI','BASIN_SP','BASIN_WP']
X = df[column_names][df['in_pre'] == True]
y = df['max_64'][df['in_pre'] == True]

In [136]:
df['dist'] = df['dist'].fillna(method='ffill')

In [137]:
y_pred_64 = rf_random_64.predict(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [125]:
X.loc[:,'Outcome'] = y_pred_64
X.to_csv('max_64_.csv')
len(X)

ValueError: Must have equal len keys and value when setting with an iterable

In [42]:
y_pred_64 = rf_random_64.predict(X)

In [43]:
df_rf_64 = pd.DataFrame({'Actual': y, 'Predicted': y_pred_64})

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y, y_pred_64))  
print('Mean Squared Error:', mean_squared_error(y, y_pred_64))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, y_pred_64)))

##PREDICTIONS

In [111]:
df_to_pred = df[df['max_64']!=df['max_64']][df['SID']==df['SID']]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [112]:
df_to_pred['dist'] = df_to_pred['dist'].fillna(method = 'ffill')

In [113]:
df['BASIN_NI']

0         1
1         1
2         1
3         1
4         1
         ..
693177    0
693178    0
693179    0
693180    0
693181    0
Name: BASIN_NI, Length: 693182, dtype: uint8

In [138]:

import numpy as np
from sklearn.model_selection import train_test_split
column_names = ['LAT','LON','WMO_WIND','WMO_PRES','DIST2LAND','LANDFALL','STORM_SPD',	
                'STORM_DR', 'dist',	
                'month_1.0',	'month_2.0',	'month_3.0',	'month_4.0',	'month_5.0',	'month_6.0',
                'month_7.0',	'month_8.0',	'month_9.0',	'month_10.0',	'month_11.0',	'month_12.0',
                'rank', 'rank_1',
                'BASIN_EP','BASIN_NI','BASIN_SI','BASIN_SP','BASIN_WP']
X = df[column_names][df['in_pre'] == True][df['LAT'] == df['LAT']]
y = df['max_64'][df['in_pre'] == True][df['LAT'] == df['LAT']]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [139]:
X

,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,LANDFALL,STORM_SPD,STORM_DR,dist,month_1.0,...,month_10.0,month_11.0,month_12.0,rank,rank_1,BASIN_EP,BASIN_NI,BASIN_SI,BASIN_SP,BASIN_WP
223479,6.60,145.00,45.0,994.0,991.0,991.0,13.0,270.0,0.680000,0,...,0,0,0,98.0,1.0,0,0,0,0,1
223480,6.60,144.32,45.0,994.0,1007.0,1007.0,13.0,270.0,0.620000,0,...,0,0,0,97.0,2.0,0,0,0,0,1
223481,6.60,143.70,45.0,994.0,1026.0,1026.0,11.0,270.0,0.530000,0,...,0,0,0,96.0,3.0,0,0,0,0,1
223482,6.60,143.17,45.0,994.0,1045.0,1040.0,10.0,270.0,0.470000,0,...,0,0,0,95.0,4.0,0,0,0,0,1
223483,6.60,142.70,45.0,994.0,1040.0,1022.0,9.0,270.0,0.450000,0,...,0,0,0,94.0,5.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693098,17.96,84.86,45.0,994.0,84.0,69.0,7.0,15.0,0.367696,0,...,0,0,0,5.0,46.0,0,1,0,0,0
693099,18.30,85.00,45.0,994.0,69.0,61.0,7.0,30.0,0.392046,0,...,0,0,0,4.0,47.0,0,1,0,0,0
693100,18.61,85.24,45.0,994.0,67.0,64.0,8.0,34.0,0.468722,0,...,0,0,0,3.0,48.0,0,1,0,0,0
693101,19.00,85.50,45.0,994.0,64.0,30.0,10.0,24.0,0.578705,0,...,0,0,0,2.0,49.0,0,1,0,0,0


In [132]:
y_pred_64 = rf_random_64.predict(X)

In [133]:
y_pred_64

array([36.03056802, 36.68896401, 35.45550536, ..., 31.69380502,
       30.94402489, 32.36615827])

In [134]:
X.loc[:,'Outcome'] = y_pred_64
X.to_csv('max_64_.csv')
len(X)

86169

In [118]:
#result = pd.merge(df1, df_out, how = 'outer', left_on=['name', 'year'],  right_on=['NAME', 'SEASON'])

In [119]:
result

NameError: name 'result' is not defined

In [ ]:
X.loc[:,'Outcome'] = y_pred_64
X.to_csv('max_64_.csv')
len(X)

In [ ]:
df_out_64 = pd.merge(df_to_pred, X, how = 'outer', on=['LAT','LON','WMO_WIND','WMO_PRES'] )

In [ ]:
df_out_64.to_csv('max_64.csv')